<a href="https://colab.research.google.com/github/MackQian/SocialComputingProjects/blob/master/hw2/socialcomputinghw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from google.colab import files
#upload dataset here
uploaded = files.upload()

Saving gender-classifier-DFE-791531.csv to gender-classifier-DFE-791531.csv


In [ ]:
import pandas as pd
import io
print(type(uploaded['gender-classifier-DFE-791531.csv']))
data = pd.read_csv(io.StringIO(uploaded['gender-classifier-DFE-791531.csv'].decode('ISO-8859-1')))

<class 'bytes'>


In [ ]:
data=data.drop(columns=["_unit_id","_unit_state","_golden","_trusted_judgments","_last_judgment_at","profile_yn","profile_yn:confidence","created","profile_yn_gold","profileimage","gender_gold","tweet_id","tweet_location","user_timezone","tweet_coord","tweet_created"],axis=1)

In [ ]:
print(data.dtypes)
data.head(1)

gender                object
gender:confidence    float64
description           object
fav_number             int64
link_color            object
name                  object
retweet_count          int64
sidebar_color         object
text                  object
tweet_count            int64
dtype: object


,gender,gender:confidence,description,fav_number,link_color,name,retweet_count,sidebar_color,text,tweet_count
0,male,1.0,i sing my own rhythm.,0,08C2C2,sheezy0,0,FFFFFF,Robbie E Responds To Critics After Win Against...,110964


In [ ]:
#delete non gendered rows
for index,row in data.iterrows():
  if row['gender']!='male' and row['gender']!='female':
    data.drop(index,inplace=True)

In [ ]:
#change genders to 1's and 0's
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
print(data['gender'].unique())
data['gender']=le.fit_transform(data['gender'])
data.head(10)

['male' 'female']


,gender,gender:confidence,description,fav_number,link_color,name,retweet_count,sidebar_color,text,tweet_count
0,1,1.0000,i sing my own rhythm.,0,08C2C2,sheezy0,0,FFFFFF,Robbie E Responds To Critics After Win Against...,110964
1,1,1.0000,I'm the author of novels filled with family dr...,68,0084B4,DavdBurnett,0,C0DEED,ÛÏIt felt like they were my friends and I was...,7471
2,1,0.6625,louis whining and squealing and all,7696,ABB8C2,lwtprettylaugh,1,C0DEED,i absolutely adore when louis starts the songs...,5617
3,1,1.0000,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",202,0084B4,douggarland,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,1693
4,0,1.0000,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,37318,3B94D9,WilfordGemma,0,0,Watching Neighbours on Sky+ catching up with t...,31462
5,0,1.0000,you don't know me.,3901,F5ABB5,monroevicious,0,0,"Ive seen people on the train with lamps, chair...",20036
7,1,1.0000,The secret of getting ahead is getting started.,80,0000FF,RobinMeske,0,C0DEED,Gala Bingo clubs bought for å£241m: The UK's l...,112117
8,0,1.0000,Pll Fan // Crazy about MCD // Ramen is bae,1825,9266CC,pigzilla_,0,0,@_Aphmau_ the pic defines all mcd fangirls/fan...,482
9,0,1.0000,"Renaissance art historian, University of Notti...",3115,9266CC,GabrieleNeher,0,FFFFFF,@Evielady just how lovely is the tree this yea...,26085
12,0,0.6509,Senior '16 . XI-XII-MMXIV.,3371,0084B4,kaylanabrewer_,0,FFFFFF,Just put my ass on the line for you and this i...,13585


In [ ]:
dataDict=data.to_dict()

In [ ]:
# the bulk of the processing time
maleWords=['bro','dude','he','his','him','actor', 'author', 'bachelor', 'boy', 'bridegroom', 'brother', 'conductor', 'count', 'czar', 'dad', 'daddy', 'duke', 'man', 'emperor', 'father', 'god', 'grandfather', 'heir', 'hero', 'host', 'husband', 'king', 'man', 'master', 'murderer', 'nephew', 'poet', 'policeman', 'prince', 'sir', 'son', 'uncle', 'wizard', 'waiter']
femaleWords=['dudette','sis','chick','she','hers','her','actress', 'authoress', 'spinster', 'girl', 'bride', 'sister', 'conductress', 'countess', 'czarina', 'mum','mom', 'mummy','mommy',  'duchess', 'woman', 'empress', 'mother', 'goddess', 'grandmother', 'heiress', 'heroine', 'hostess', 'wife', 'queen', 'woman', 'mistress', 'murderess', 'niece', 'poetess', 'policewoman', 'princess', 'madam', 'daughter', 'aunt', 'witch', 'waitress']
maleNames=['james', 'john', 'robert', 'michael', 'william', 'david', 'richard', 'joseph', 'thomas', 'charles', 'christopher', 'daniel', 'matthew', 'anthony', 'donald', 'mark', 'paul', 'steven', 'andrew', 'kenneth', 'joshua', 'george', 'kevin', 'brian', 'edward', 'ronald', 'timothy', 'jason', 'jeffrey', 'ryan', 'jacob', 'gary', 'nicholas', 'eric', 'stephen', 'jonathan', 'larry', 'justin', 'scott', 'brandon', 'frank', 'benjamin', 'gregory', 'samuel', 'raymond', 'patrick', 'alexander', 'jack', 'dennis', 'jerry', 'tyler', 'aaron', 'jose', 'henry', 'douglas', 'adam', 'peter', 'nathan', 'zachary', 'walter', 'kyle', 'harold', 'carl', 'jeremy', 'keith', 'roger', 'gerald', 'ethan', 'arthur', 'terry', 'christian', 'sean', 'lawrence', 'austin', 'joe', 'noah', 'jesse', 'albert', 'bryan', 'billy', 'bruce', 'willie', 'jordan', 'dylan', 'alan', 'ralph', 'gabriel', 'roy', 'juan', 'wayne', 'eugene', 'logan', 'randy', 'louis', 'russell', 'vincent', 'philip', 'bobby', 'johnny', 'bradley']
femaleNames=['mary', 'patricia', 'jennifer', 'linda', 'elizabeth', 'barbara', 'susan', 'jessica', 'sarah', 'karen', 'nancy', 'margaret', 'lisa', 'betty', 'dorothy', 'sandra', 'ashley', 'kimberly', 'donna', 'emily', 'michelle', 'carol', 'amanda', 'melissa', 'deborah', 'stephanie', 'rebecca', 'laura', 'sharon', 'cynthia', 'kathleen', 'helen', 'amy', 'shirley', 'angela', 'anna', 'brenda', 'pamela', 'nicole', 'ruth', 'katherine', 'samantha', 'christine', 'emma', 'catherine', 'debra', 'virginia', 'rachel', 'carolyn', 'janet', 'maria', 'heather', 'diane', 'julie', 'joyce', 'victoria', 'kelly', 'christina', 'joan', 'evelyn', 'lauren', 'judith', 'olivia', 'frances', 'martha', 'cheryl', 'megan', 'andrea', 'hannah', 'jacqueline', 'ann', 'jean', 'alice', 'kathryn', 'gloria', 'teresa', 'doris', 'sara', 'janice', 'julia', 'marie', 'madison', 'grace', 'judy', 'theresa', 'beverly', 'denise', 'marilyn', 'amber', 'danielle', 'abigail', 'brittany', 'rose', 'diana', 'natalie', 'sophia', 'alexis', 'lori', 'kayla', 'jane']

dataDict['maleWordCount']={}
dataDict['femaleWordCount']={}
dataDict['maleName']={}
dataDict['femaleName']={}
dataDict['textUniqueWordsCount']={}

dataDict['neg']={}
dataDict['neu']={}
dataDict['pos']={}
dataDict['compound']={}

posDict=[ 'NNS', 'JJ', 'VBP', 'VB', 'VBD', 'VBG', 'PRP$', 'NN', 'PRP', 'RB', 'VBZ', 'VBN', 'CD', 'JJS', 'RBR', 'JJR', 'NNP', 'WP', 'RBS', 'UH', 'NNPS', 'WP$']

for x in posDict:
  dataDict[x]={}

from sklearn.feature_extraction.text import CountVectorizer

import nltk
nltk.download('averaged_perceptron_tagger')
vectorizer = CountVectorizer(strip_accents='unicode')


nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer
for index in dataDict['description'].keys():
  for x in posDict:
    dataDict[x][index]=0
  try:
    vectorizer.fit_transform([dataDict['description'][index]])
    count=0
    for w in maleWords:
      if w in vectorizer.get_feature_names():
        count+=1
    dataDict['maleWordCount'][index]=count

    count=0
    for w in femaleWords:
      if w in vectorizer.get_feature_names():
        count+=1
    dataDict['femaleWordCount'][index]=count
  except ValueError:
    dataDict['femaleWordCount'][index]=0
    dataDict['maleWordCount'][index]=0

  vectorizer.fit_transform([dataDict['name'][index]])
  
  count=0
  for w in maleNames:
    if w in vectorizer.get_feature_names():
      count+=1
  dataDict['maleName'][index]=count

  count=0
  for w in femaleNames:
    if w in vectorizer.get_feature_names():
      count+=1
  dataDict['femaleName'][index]=count
  
  words=dataDict['text'][index].split()
  newWords=[]
  for w in words:
    if not 't.co/' in w:
      newWords.append(w)
  sentence=" ".join(newWords)
  vectorizer.fit_transform([sentence])
  dataDict['textUniqueWordsCount'][index]=len(vectorizer.get_feature_names())
  y=nltk.pos_tag(vectorizer.get_feature_names())
  for thing in y:
    if thing[1] in posDict:
      dataDict[thing[1]][index]+=1
  sid = SentimentIntensityAnalyzer()
  x=sid.polarity_scores(sentence)
  for z in x.keys():
    dataDict[z][index]=x[z]
  
  

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
data2=pd.DataFrame()
data2=data2.from_dict(dataDict)

In [ ]:
data2=data2.drop(columns=['description','fav_number','link_color','name','retweet_count','sidebar_color','text','gender:confidence','tweet_count'],axis=1)
data2.head()

,gender,maleWordCount,femaleWordCount,maleName,femaleName,textUniqueWordsCount,neg,neu,pos,compound,NNS,JJ,VBP,VB,VBD,VBG,PRP$,NN,PRP,RB,VBZ,VBN,CD,JJS,RBR,JJR,NNP,WP,RBS,UH,NNPS,WP$
0,1,0,0,0,0,12,0.138,0.625,0.237,0.3818,4,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,16,0.000,0.714,0.286,0.6808,1,1,2,0,3,1,1,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,14,0.062,0.615,0.323,0.7267,3,2,0,0,1,0,0,0,2,2,1,0,0,0,0,0,0,0,0,0,0,0
3,1,3,0,0,0,18,0.000,0.874,0.126,0.3786,2,2,2,2,1,1,0,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,14,0.000,1.000,0.000,0.0000,1,2,1,0,0,2,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
target=data2['gender']
data2=data2.drop(columns=['gender'],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
data_train, data_test, target_train, target_test = train_test_split(data2, target, test_size = 0.30, random_state = 10)

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score
svc_model = LinearSVC()
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score)}
kfold=cross_validate(svc_model, data2, target, cv=10,scoring=scoring)
for x in kfold.keys():
  print(x,end=": ")
  print(kfold[x])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

fit_time: [1.20264769 1.23569679 1.24640489 1.23418546 1.23487782 1.23097539
 1.21651626 1.23022246 1.23690462 1.23056746]
score_time: [0.00676751 0.00654197 0.00849128 0.00722122 0.00575566 0.00588894
 0.00533247 0.00590801 0.0054369  0.0059092 ]
test_accuracy: [0.48682171 0.48449612 0.55658915 0.52093023 0.48642358 0.4996121
 0.48719938 0.55314197 0.52831652 0.52288596]
test_precision: [0.48238255 0.48169243 0.56521739 0.66666667 0.48286853 0.48828829
 0.48164336 0.58431373 0.51957295 0.55263158]
test_recall: [0.92741935 0.95483871 0.33548387 0.00645161 0.97899838 0.87560582
 0.8901454  0.24071082 0.2358643  0.03392569]


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
uploaded = files.upload()

Saving Mar07_GroupB.txt to Mar07_GroupB (1).txt


In [ ]:
f=open("Mar07_GroupB.txt","r")
fusers=['julie', 'eva', 'rebecca', 'cinthia', 'caroline','ariel', 'lynn', 'judith']
musers=['josh', 'jake', 'matthew', 'luke', 'aaron', 'ted']

conv={}
conv['gender']={}
conv['maleWordCount']={}
conv['femaleWordCount']={}
conv['maleName']={}
conv['femaleName']={}
conv['textUniqueWordsCount']={}
conv['neg']={}
conv['neu']={}
conv['pos']={}
conv['compound']={}
index=0
for x in posDict:
      conv[x]={}
for line in f:
    x=line.split("):")
    u=x[0].split()[0]
    if u=="moderator":
      continue
    if u in fusers:
      conv['gender'][index]=0
    else:
      conv['gender'][index]=1
    if u in maleNames:
      conv['maleName'][index]=1
    else:
      conv['maleName'][index]=0
    if u in femaleNames:
      conv['femaleName'][index]=1
    else:
      conv['femaleName'][index]=0
    conv['maleWordCount']=0
    conv['femaleWordCount']=0
    for pos in posDict:
      conv[pos][index]=0
    sentence=x[1].strip()
    try:
      vectorizer.fit_transform([sentence])
      conv['textUniqueWordsCount'][index]=len(vectorizer.get_feature_names())
      y=nltk.pos_tag(vectorizer.get_feature_names())
      for thing in y:
        if thing[1] in posDict:
          conv[thing[1]][index]+=1
      sid = SentimentIntensityAnalyzer()
      m=sid.polarity_scores(sentence)
      for z in m.keys():
        conv[z][index]=m[z]
    except ValueError:
      conv['textUniqueWordsCount'][index]=0
      conv['neg'][index]=0
      conv['neu'][index]=0
      conv['pos'][index]=0
      conv['compound'][index]=0
    index+=1
convData=pd.DataFrame()
convData=convData.from_dict(conv)

In [ ]:
convData.tail()
convTarget=convData['gender']
convData=convData.drop(columns=['gender'],axis=1)

In [ ]:
#data2=data2.drop(columns=['gender'],axis=1)
data2.head()

,maleWordCount,femaleWordCount,maleName,femaleName,textUniqueWordsCount,neg,neu,pos,compound,NNS,JJ,VBP,VB,VBD,VBG,PRP$,NN,PRP,RB,VBZ,VBN,CD,JJS,RBR,JJR,NNP,WP,RBS,UH,NNPS,WP$
0,0,0,0,0,12,0.138,0.625,0.237,0.3818,4,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,16,0.000,0.714,0.286,0.6808,1,1,2,0,3,1,1,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,14,0.062,0.615,0.323,0.7267,3,2,0,0,1,0,0,0,2,2,1,0,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,18,0.000,0.874,0.126,0.3786,2,2,2,2,1,1,0,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,14,0.000,1.000,0.000,0.0000,1,2,1,0,0,2,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
acc=0.0
prec=0.0
rec=0.0
for i in range(10):
  pred = svc_model.fit(data2, target).predict(convData)
  acc+=accuracy_score(convTarget, pred, normalize = True)
  prec+=precision_score(convTarget, pred)
  rec+=recall_score(convTarget, pred)
print(acc/10)
print(prec/10)
print(rec/10)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to co

0.52576112412178
0.2439570330525509
0.04870466321243523


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
